### There should be three evaluation metrics:
* semantic similarity between val and gen. adv. caption for meaningfulness
* verification for removal of Target Class in gen. adv. caption
* verification for retaining of Correct Class in gen. adv. caption

In [1]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

import string
import spacy
from spacy.lang.en import stop_words
nlp = spacy.load("en_core_web_sm")

import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")
ps = PorterStemmer()

from gensim.models import Word2Vec, KeyedVectors
import gensim
import numpy as np
from gensim.models import KeyedVectors

from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\75y\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv(r"dataset_csv 1.csv")

df[['Target Class', 'Correct Class']]

,Target Class,Correct Class
0,dog,skateboard
1,frisbee,person
2,bicycle,people
3,people,elephant
4,teddy bear,plate
5,skis,person
6,flowers,vase


In [4]:
def object_removal_success_rate(sent:str,
                                target_cls:str,
                                correct_cls:str,
                                sim_threshold:float,
                                which_embedding:str,
                                model,
                                show_cosineSimilarity=False) -> [int,int]:
    """
    input: Generated Adv Caption, Target Class, Correct Class
    output: tuple of binary scores (removed/not, retained/not)
    """

    # word tokenization
    words = [str(word) for word in nlp(sent)]
    # remove stopwords
    words = list(set([w for w in words if w not in stop_words.STOP_WORDS]))
    # remove punctuations
    words = [w for w in words if w not in set(string.punctuation)]
    # lemmatize words
    words = [nlp(w)[0].lemma_ for w in words]
    # filter on certain POS
    words = [w for w in words if all(nlp(w)[0].pos_!=p for p in ['DET','PRON'])]
    
    # check if all words exists in model vocab. if not, then use stemming to get the root form 
    words = [w if w in model else ps.stem(w) for w in words]
    
    # get word-embeddings for all words in the gen. adv. caption (i.e., candidate words),.
    # target class, and correct class
    word_vec    = {word: model[word].reshape(1, -1) for word in words}
    
    if target_cls not in model: target_vec = np.mean([model[str(w)] for w in nlp(target_cls)], axis=0).reshape(1, -1)
    else: target_vec = model[target_cls].reshape(1, -1)
        
    correct_vec = model[correct_cls].reshape(1, -1)

    # check if a candidate word has high similarity with the target class but is a verb (e.g., lay & people). 
    # if so, then remove that word.
    word_vec = {w:v for w,v in word_vec.items() 
                if not (cosine_similarity(v,target_vec)[0][0] > sim_threshold and nlp(w)[0].pos_=='VERB')}
    
    # calculate cosine similarity between candidate words and target class;
    # check if all of the similarity scores is lower than a threshold.
    # if yes, then consider that a success in target object removal and assign score 1.
    target_class_removed_success = all([1 if cosine_similarity(v,target_vec)[0][0] < sim_threshold else 0
                                        for k,v in word_vec.items()])
    
    # calculate cosine similarity between candidate words and correct class;
    # check if any of the similarity scores is higher than a threshold.
    # if yes, then consider that a success in correct object retention and assign score 1.
    correct_class_retained_success = any([1 if cosine_similarity(v,correct_vec)[0][0] > sim_threshold else 0
                                        for k,v in word_vec.items()])

    if show_cosineSimilarity:
        for k,v in word_vec.items():
            print(k)
            print(f"Similarity with target {target_cls}  : {cosine_similarity(v,target_vec)[0][0]}")
            print(f"Similarity with correct {correct_cls}: {cosine_similarity(v,correct_vec)[0][0]}")
            print("="*50)
        print("*"*50)
    
    return [target_class_removed_success, correct_class_retained_success]


def load_embedding_model(which_embedding:str):
    """
    this function loads the word embedding model
    """
    # (can add more options -- fastext, bert)
    if which_embedding=='word2vec':
        model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    
    elif which_embedding=='glove':
        def load_glove_model(glove_file):
            with open(glove_file, 'r', encoding='utf-8') as f:
                word_vectors = {}
                for line in f:
                    split_line = line.split()
                    word = split_line[0]
                    vector = np.array(split_line[1:], dtype='float32')
                    word_vectors[word] = vector
            return word_vectors

        dim = 50
        glove_model = load_glove_model(rf'glove.6B\glove.6B.{dim}d.txt')
        model = KeyedVectors(vector_size=dim)
        model.add_vectors(list(glove_model.keys()), list(glove_model.values()))

    return model


In [5]:
which_embedding = 'glove'
model = load_embedding_model(which_embedding)

In [6]:
df['Score'] = df.progress_apply(lambda x:object_removal_success_rate(x['Generated Adv Caption (Captions)'],
                                                                     x['Target Class'],
                                                                     x['Correct Class'],
                                                                     0.7, which_embedding, model,
                                                                     show_cosineSimilarity=False
                                                                    ), axis=1)

100%|██████████| 7/7 [00:01<00:00,  6.06it/s]


In [7]:
df[['Attack Success (Target Class)', 'Attack Success (Correct Class)']] = pd.DataFrame(df['Score'].tolist(), index=df.index)
df = df.drop(columns=['Score'])

In [8]:
df

,ID,COCO Classes,Target Class,Correct Class,Correct COCO Caption,Trained Adv Caption,Validation Adv Caption,Generated Orig Caption,Generated Adv Caption (Classes),Generated Adv Caption (Captions),...,Cosine Sim of Adv Image & Trained Adv Caption,Cosine Sim of Adv Image & Validation Adv Caption,Baseline (Cosine Sim of Trained & Validation Adv Captions),Cosine Sim of Generated Adv Caption & Generated Orig Caption,Cosine Sim of Generated Adv Caption & Trained Adv Caption,Cosine Sim of Generated Adv Caption & Validation Adv Caption,Training Loss,Validation Loss,Attack Success (Target Class),Attack Success (Correct Class)
0,490,"['dog', 'skateboard']",dog,skateboard,A dog stands next to a skateboard leaned up ag...,A skateboard leaned up against the wall.,There is a skateboard next to the wall.,A dog standing next to a skateboard on a hard ...,A person standing next to a skateboard on a fl...,A skateboard sticking out of a wall next to a ...,...,0.5015,0.4512,0.9448,0.6631,0.9458,0.9102,0.2827,0.2471,True,True
1,12501,"['person', 'frisbee']",frisbee,person,A group of people playing a game with frisbee ...,A group of people are outside on the grass.,People stand in the grassy field outside.,A group of people playing frisbee in a park.,A woman holding a handbag next to a woman hold...,A group of people in grassy field next to trees.,...,0.4746,0.4062,0.9097,0.7217,0.8823,0.8765,0.1606,0.1548,True,True
2,1053,"['bicycle', 'people', 'traffic light']",bicycle,people,A group of bicycle riders biking across a pave...,A group of people are walking across a paved s...,A group of people crossed a paved street.,A group of people riding bikes down a street.,A group of people crossing a street in the rain.,A group of people walking across a street.,...,0.5352,0.4524,0.9058,0.8657,0.9175,0.8945,0.0518,0.0288,True,True
3,1164,"['people', 'elephant']",people,elephant,Two girls bathe an elephant lying on its side.,An elephant lays on its side in a river.,"In the shallow water, the elephant lays down.",A couple of people washing an elephant in a ri...,An elephant that is standing in the water.,An elephant that is laying down in the water.,...,0.5176,0.3872,0.7549,0.8037,0.9004,0.8726,0.0967,0.0688,True,True
4,438701,"['person', 'plate', 'teddy bear']",teddy bear,plate,A small boy sitting next to a bear eating dinner,A boy has a plate of food in front of him.,A young boy is eating a plate of food.,A boy sitting at a table with a teddy bear.,A woman is sitting in front of a plate of food.,A boy sitting at a table with plates of food.,...,0.4805,0.4512,0.9043,0.7876,0.8711,0.8813,0.0835,0.0938,True,True
5,10024,"['person', 'skis']",skis,person,A man walking through a snow covered forest on...,A man walking through a snow covered forest.,"On the snowy trail in the woods, the man walks.",A man riding skis across a snow covered forest.,A person with an umbrella standing on a walkway.,A man walking through a forest covered in snow.,...,0.5488,0.4172,0.7905,0.8584,0.9814,0.8047,0.1230,-0.0537,True,True
6,16890,"['flowers', 'vase']",flowers,vase,A vase of flowers sitting on a stand in front ...,A vase is sitting on a stand in front of a red...,"On the table is an old, teal ceramic vase.",A vase filled with purple flowers on top of a ...,A vase sitting on top of a table covered in fl...,A vase sitting on top of a table next to a red...,...,0.5923,0.3577,0.5933,0.7598,0.8159,0.6738,0.0562,-0.0859,False,True


In [9]:
# df.to_csv(...., index=False)